# Análise dos produtos comercializados

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = APIKEYHERE
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


In [3]:
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    APIKEY = config['comtrade']['key']

comtrade.init(APIKEY)

## China, categorias de produtos mais importantes nas trocas com os PLP

Obtem os totais agregados com nível 2 de código HS e lista os primeiros.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx


### TODO

- [ ] Permitir escolher os reporter e partners
- [ ] compatibilizar com o formato de report da AICEP https://myaicep.portugalexporta.com/mercados-internacionais/cn/china?setorProduto=-1
- [ ] há problemas de duplicação de linhas nas listagem de detalhes de commodities, provavlmente por custom code, ou mot code.
  

In [4]:
import comtrade
import pandas as pd


rank_filter = 5  # número de importações mais relevantes

year_start = 1993
year_end = 2023
years = comtrade.year_range(year_start,year_end)
reporterCode = comtrade.m49_china
partnerCode = comtrade.m49_plp_list
flowCode = 'M,X'
pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
            'flowCode','primaryValue']



pco_top5_sorted = comtrade.top_commodities(reporterCode, partnerCode,  years, flowCode,rank_filter, pco_cols)

# save to Excel
# Prepare file name for Excel output
filename_note=f"{year_start}-{year_end}_{flowCode.replace(',','_')}"  # change to append to filename
excel_file_name = f"./reports/china_plp_top5_{filename_note}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name)

# Save data
sheet_name = "comtrade"
pco_top5_sorted.to_excel(excel_file, sheet_name=sheet_name, index=False, startrow=2)
# Adjust column width
comtrade.excel_col_autowidth(pco_top5_sorted, excel_file,sheet_name)

# format column primaryValue as currency
comtrade.excel_format_currency(pco_top5_sorted, excel_file,sheet_name, columns=['primaryValue'],width=20)
    
# Write title in first row TBD
excel_file_title = f"Trocas Comerciais China-PLP, top {rank_filter} produtos (M=Importações, X=Exportações), {years}, valores USD"
excel_file.sheets[sheet_name].write(0, 0, excel_file_title)
print("Guardado em:",excel_file_name)

excel_file.close()
# show
pd.options.display.max_colwidth=100
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 100
pco_top5_sorted.head(50)

Guardado em: ./reports/china_plp_top5_1993-2023_M_X.xlsx


cmdCode  \
reporterDesc partnerDesc flowCode refYear rank           
China        Angola      M        1993    1         27   
                                          2         84   
                                          3         85   
                                          4         48   
                                  1994    1         27   
                                          2         71   
                                          3         84   
                                          4         39   
                                          5         34   
                                  1995    1         27   
                                          2         84   
                                          3         71   
                                          4         51   
                                          5         99   
                                  1996    1         27   
                                          2         03   
                                          3         55   
                                          4         71   
                                          5         39   
                                  1997    1         27   
                                          2         39   
                                          3         03   
                                          4         90   
                                  1998    1         27   
                                          2         03   
                                          3         71   
                                          4         84   
                                  1999    1         27   
                                          2         84   
                                          3         85   
                                          4         82   
                                  2000    1         27   
                                          2         44   
                                          3         73   
                                          4         39   
                                          5         85   
                                  2001    1         27   
                                          2         44   
                                          3         03   
                                          4         25   
                                          5         88   
                                  2002    1         27   
                                          2         03   
                                          3         44   
                                          4         16   
                                          5         83   
                                  2003    1         27   
                                          2         25   
                                          3         71   
                                          4         39   

                                                                                                                                            cmdDesc  \
reporterDesc partnerDesc flowCode refYear rank                                                                                                        
China        Angola      M        1993    1     Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral w...   
                                          2                           Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof   
                                          3     Electrical machinery and equipment and parts thereof; sound recorders and reproducers; televisio...   
                                          4                                    Paper and paperboard; articles of paper pulp, of paper or paperboard   
                                  1994    1     Mineral fuels, mineral oils and products of their distillation; bituminous s

## Análise de variações nos produtos mais importantes

A célula seguinte produz uma lista das variações mais importantes na evolução das 
trocas comerciais entre a China e os Países de Língua Portuguesa

Por cada combinação de países e de fluxo (M=importações pela China, X=exportações da China)
por exemplo:

        China Brazil M

        1993   72,26
                -> 26   141,782,165.00 Ores, slag and ash
                -> 72   540,603,898.00 Iron and steel
        1994 ! 15,72
                -> 15   463,690,083.00 Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes
        1995 ! 15,26
        1996 ! 15,23
                -> 23   270,712,928.00 Food industries, residues and wastes thereof; prepared animal fodder
        1997 ≈ 23,15
        1998 ! 26,23
        1999 ! 26,12
                -> 12   171,963,034.00 Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit, industrial or medicinal plants; straw and fodder
        2000 ≈ 12,26
        2001 ≈ 26,12
        2002 ≈ 12,26
        2003 =
        2004 ≈ 26,12

Cada ano é seguido de um sinal que indica a mudança ocorrida em relação ao ano anterior:
* ! : mudança num dos produtos principais
* = : mesmos produtos do ano anterior, pela mesma ordem de importância
* ≈ : mesmos produtos do ano anterior, mas importância diferente


Cada vez que um novo produto aparece na série é fornecida a descrição do código respectivo e o valor envolvido.

Esta listagem pode ser produzida tendo em conta um número variáveis de produtos principais
normalmente os dois principais (ver variável `consider_first`)

In [5]:
consider_first = 2
for reporter in pco_top5_sorted.index.get_level_values(0).unique():
    for partner in pco_top5_sorted.loc[(reporter,)].index.get_level_values(0).unique(): 
        for flow in pco_top5_sorted.loc[((reporter,partner,))].index.get_level_values(0).unique():
            print(reporter,partner, flow)
            previous_pattern = []
            all_commodities = set() # all commodities for this partner for this flow
            all_commodities_desc = set() # all commodities for this partner for this flow descriptions
            previous_years_values = dict()
            for year in pco_top5_sorted.loc[(reporter,partner,flow,)].index.get_level_values(0).unique():
                print(year, end=' ')
                top5_codes = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdCode'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_codes_desc = pco_top5_sorted.loc[(reporter,partner,flow,year)]['cmdDesc'].astype(str).values.flatten().tolist()[0:consider_first]
                top5_primary_value = pco_top5_sorted.loc[(reporter,partner,flow,year)]['primaryValue'].values.flatten().tolist()[0:consider_first]
                top5_table = {code: {"desc":desc, "value":value, "year": year} for code,desc,value in zip(top5_codes,top5_codes_desc,top5_primary_value)}
                codes_desc = {f"{code} {desc}" for code,desc in zip(top5_codes,top5_codes_desc)}
                # make set of tuples top5_codes and top5_codes_desc
                new_commodities = set(top5_codes).difference(all_commodities)
                all_commodities = all_commodities.union(set(top5_codes))
                all_commodities_desc = all_commodities_desc.union(codes_desc)
                pattern = ','.join(top5_codes)
                if len(previous_pattern) == 0:
                    previous_pattern = top5_codes
                    # update previous_years_values with top5_table
                    previous_years_values.update(top5_table)
                    print(' ', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                    for cmd in new_commodities:
                        print( "        ->", cmd,top5_table[cmd]['desc'])
                elif set(top5_codes) == set(previous_pattern):
                    if top5_codes == previous_pattern:
                        print('=', pattern, end=' | ')
                    else:
                        print('≈', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                    print()
                else:
                    print('!', pattern, end=' | ')
                    for cmd in top5_codes:
                        print(f"{top5_table[cmd]['value']:,.2f} ", end=' ')
                        if cmd in previous_years_values.keys():
                            print(f"({top5_table[cmd]['value'] / previous_years_values[cmd]['value']:.0%})", end=' | ')
                        else:
                            print("(new)", end=' | ')
                    print()
                    if len(new_commodities) > 0:
                        for cmd in new_commodities:
                            print( "        ->", cmd, top5_table[cmd]['desc'])
                       
                previous_pattern = top5_codes
                previous_years_values.update(top5_table)
            for cmd in sorted(all_commodities_desc):
                print( "    ",cmd)
            print(80*"-")



China Angola M
1993   27,84 | 179,857,111.00 (100%) | 839,509.00 (100%) | 
        -> 27 Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes
        -> 84 Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof
1994 ! 27,71 | 47,971,662.00  (27%) | 9,835.00  (new) | 
        -> 71 Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin
1995 ! 27,84 | 135,763,069.00  (283%) | 1,012,594.00  (121%) | 
1996 ! 27,03 | 243,536,861.00  (179%) | 131,488.00  (new) | 
        -> 03 Fish and crustaceans, molluscs and other aquatic invertebrates
1997 ! 27,39 | 603,131,937.00  (248%) | 209,792.00  (new) | 
        -> 39 Plastics and articles thereof
1998 ! 27,03 | 153,453,056.00  (25%) | 237,697.00  (181%) | 
1999 ! 27,84 | 355,615,886.00  (232%) | 20,000.00  (2%) | 
2000 ! 27,44 | 1,842,016,530.00  (518%) | 671,664.00  (new) | 
 

## Detalhe do tipo de produtos comercializados entre os PLP e a China

Em dois passos: 
* obtém as categorias de nível 2 mais importantes de cada país
* pesquisa todos as subcategorias de cada um das categorias

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [6]:

import ipywidgets as widgets
from IPython.display import display
import comtrade

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = comtrade.m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

years = widgets.SelectMultiple(
    options=range(2000,2022),
    value=[2021],
    description='Ano:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Fonte (reporter):',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=[("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE ,
    # value=[("Angola",m49_angola)],
    description='Parceiro:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [7]:

display(years)
display(reporterCodeWidget)
display(partnerCodeWidget)
display(flowCodeWidget)


SelectMultiple(description='Ano:', index=(21,), options=(2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,…

Dropdown(description='Fonte (reporter):', options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Angol…

Dropdown(description='Parceiro:', options=(('China', 156), ('Macau', 446), ('Hong Kong', 344), ('Angola', 24),…

Dropdown(description='Fluxo:', options=(('Importações', 'M'), ('Exportações', 'X')), value='M')

Depois de escolher ano, país e fluxo executar a célula seguinte.

Para significado detalhado de cada código ver: https://www.wcoomd.org/en/topics/nomenclature/instrument-and-tools/hs-nomenclature-2022-edition/hs-nomenclature-2022-edition.aspx

In [8]:
import comtrade
from comtrade import COUNTRY_CODES, HS_CODES, HS_CODES_DF, HS_CODES_L2_DF


yearsList = ",".join(list(map(str,years.value)))
partnerCode = partnerCodeWidget.value
reporterCode = reporterCodeWidget.value


# decode country
partner_name = comtrade.COUNTRY_CODES.get(partnerCode)
reporter_name = comtrade.COUNTRY_CODES.get(reporterCode)

flow = flowCodeWidget.value
print(reporter_name,yearsList,flow,partner_name,)

pco_cols = ['reporterDesc','partnerDesc','refYear','rank','cmdDesc',
            'flowCode','primaryValueFormated']

pco_cols_detail = ['reporterDesc','partnerDesc','refYear','cmdCodeAG2','cmdCode','cmdDesc',
            'flowCode','primaryValue', 'isAggregate']

# first we collect the top commodity level 2 codes            
df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     cmdCode="AG2",
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     period=yearsList,
                     timeout=30,
                     echo_url=True
                     )

pco = df.sort_values(['partnerDesc','refYear','primaryValue'], ascending=[True,True,False])
pco['rank'] = pco.groupby(['partnerDesc','refYear'])["primaryValue"].rank(method="dense", ascending=False)
pco_top5 = pco[pco['rank'] <= rank_filter]
# get the countries
countries = pco_top5.partnerDesc.unique()
country_cmd_top5_codes = dict()

# get the detailed commodity codes for the top of each country
for country in countries:
    l2_codes = pco_top5[pco_top5.partnerDesc == country]['cmdCode'].unique()
    print(country,l2_codes)
    hs_details = []
    for l2_code in l2_codes:
        l2_sub_codes = list(HS_CODES_DF[HS_CODES_DF.hscode.str.startswith(l2_code)]['hscode'])
        hs_details = hs_details + l2_sub_codes
    # print(hs_details)
    country_cmd_top5_codes[country] = hs_details.copy()

# now we fetch the detail
countryDesc = COUNTRY_CODES[partnerCode]
country_cmd_codes = ",".join(country_cmd_top5_codes[countryDesc])
print(country_cmd_codes)

df = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flow,
                     #cmdCode=country_cmd_codes, # sometimes the detail commodity codes generates errors
                     cmdCode=None, # This gives all the commodities for the country and year 
                     reporterCode=reporterCode,
                     partnerCode=partnerCode,
                     partner2Code=0,
                     period=yearsList,
                     echo_url=True,
                     timeout=60
                     )
                     
df['cmdCodeAG2'] = df.cmdCode.str[0:2]
# filter the detail commodity codes by the top 5
df = df[df['cmdCodeAG2'].isin(country_cmd_top5_codes[countryDesc])]

China 2021 M China
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2021&partnerCode=156&partner2Code=0&cmdCode=AG2&flowCode=M&customsCode=C00&subscription-key=HIDDEN
China ['85' '84' '90' '89' '39']
85,8501,850110,850120,850131,850132,850133,850134,850140,850151,850152,850153,850161,850162,850163,850164,850171,850172,850180,8502,850211,850212,850213,850220,850231,850239,850240,8503,850300,8504,850410,850421,850422,850423,850431,850432,850433,850434,850440,850450,850490,8505,850511,850519,850520,850590,8506,850610,850630,850640,850650,850660,850680,850690,8507,850710,850720,850730,850750,850760,850780,850790,8508,850811,850819,850860,850870,8509,850940,850980,850990,8510,851010,851020,851030,851090,8511,851110,851120,851130,851140,851150,851180,851190,8512,851210,851220,851230,851240,851290,8513,851310,851390,8514,851411,851419,851420,851431,851432,851439,851440,851490,8515,851511,851519,851521,851529,851531,851539,851580,851590,8516,851610,851621,851629,851631,85

Temos que identificar as linhas com valor agregado porque
os dados não tem essa coluna correcta

In [9]:
# set the index to reporterDesc, partnerDesc, refYear, cmdCodeAG2
df.sort_values(['reporterDesc','partnerDesc','refYear','flowCode','cmdCode'], inplace=True)
# print(df2.info())

# list unique values of MultiIndex
lastCode = "---"
lastIndex = 0
for row in df.iterrows():
    currentCode = row[1]['cmdCode']
    if currentCode != lastCode and currentCode.startswith(lastCode):
        # print(f">>>> Last code {lastCode} index {lastIndex} is parent of {currentCode}")
        df.loc[lastIndex,'isAggregate'] = True
    # print(df.loc[row[0]][['cmdCode','cmdDesc']])
    lastCode = currentCode
    lastIndex = row[0]
df['valueForTotal'] = df['primaryValue']
df.loc[df['isAggregate'] == True,'valueForTotal'] = 0
df['subtotalAG2'] = df.groupby(['partnerCode','refYear','flowCode','cmdCodeAG2'])["valueForTotal"].transform('sum')
df['percentAG2'] = df['primaryValue'] / df['subtotalAG2'] * 100

In [11]:
import comtrade

excel_file_name = f"./reports/product_detail_{reporter_name}_{yearsList}_{flow}_{partner_name}.xlsx"
excel_file = pd.ExcelWriter(excel_file_name, engine='xlsxwriter')

df2 = df[pco_cols_detail+['percentAG2']].sort_values('primaryValue',ascending=False)
df2.to_excel(excel_file, sheet_name='detail', index=False, startrow=2)

comtrade.excel_col_autowidth(df2, excel_file, 'detail')
comtrade.excel_format_currency(df2, excel_file,'detail', columns=['primaryValue'],width=20)
excel_file.close()
print(f"Excel file {excel_file_name} created")

pd.options.display.max_rows = 500
# df.sort_values(['partnerDesc','refYear','flowCode','subtotalAG2','cmdCodeAG2','primaryValue'],ascending=[True,True,True,False,True,False])[pco_cols_detail].head(500)
df.sort_values(['reporterDesc','partnerDesc','refYear','subtotalAG2','cmdCode'],ascending=[True,True,True,False,True])[pco_cols_detail+['percentAG2']].head(500)
